# Установка библиотек и импорты

Удаляем albumentations и ultralytics для предотвращения ошибки.

In [ ]:
!pip uninstall ultralytics
!pip uninstall albumentations -y

Удаляем папки, созданные в предыдущих запусках

In [ ]:
# !rm -rf "/kaggle/working/DocLayout YOLO"
# !rm -rf /kaggle/working/wandb
# !rm -rf /kaggle/working/config

Скачиваем библиотеки для работы DocLayout-YOLO

In [ ]:
!pip install doclayout_yolo
!pip install huggingface_hub

Пишем стандартные импорты

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
from pathlib import Path
from doclayout_yolo import YOLOv10
from huggingface_hub import hf_hub_download
import json
import shutil
import cv2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Настраиваем wandb

In [ ]:
!pip install wandb --q

import wandb

secret_value = 'тут ключ от wandbai'

wandb.login(key=secret_value)


# Подготовка датасета

Создаём необходимые репозитории:

In [ ]:
!mkdir /kaggle/working/datasets

In [ ]:
!mkdir /kaggle/working/datasets/prepared_data

In [ ]:
!mkdir /kaggle/working/config

Переводим датасет в YOLO формат

In [ ]:
def convert_to_yolo_format(bbox, img_width, img_height):
    """Конвертация координат из абсолютных в относительные для YOLO"""
    x_center = (bbox[0] + bbox[2]) / 2 / img_width
    y_center = (bbox[1] + bbox[3]) / 2 / img_height
    width = (bbox[2] - bbox[0]) / img_width
    height = (bbox[3] - bbox[1]) / img_height
    return [x_center, y_center, width, height]

def prepare_dataset():
    """Подготовка датасета"""
    raw_data_dir = Path('/kaggle/input/doclayout-raw-data/raw_data')
    output_dir = Path('/kaggle/working/datasets/prepared_data')
    
    print(f"Проверка директорий:")
    print(f"raw_data_dir: {raw_data_dir}")
    print(f"output_dir: {output_dir}")

    # проверяем существование директорий
    if not (raw_data_dir / "images").exists():
        raise ValueError(f"Директория с изображениями не найдена: {raw_data_dir / 'images'}")
    if not (raw_data_dir / "jsons").exists():
        raise ValueError(f"Директория с JSON файлами не найдена: {raw_data_dir / 'jsons'}")

    # создаем необходимые директории
    images_dir = output_dir / "images"
    labels_dir = output_dir / "labels"
    images_dir.mkdir(parents=True, exist_ok=True)
    labels_dir.mkdir(parents=True, exist_ok=True)

    # словарь для маппинга классов
    class_map = {
        "title": 0, "paragraph": 1, "table": 2, "picture": 3,
        "table_signature": 4, "picture_signature": 5, "numbered_list": 6,
        "marked_list": 7, "header": 8, "footer": 9, "footnote": 10,
        "formula": 11
    }

    # собираем пары файлов
    valid_pairs = []
    image_files = list(Path(raw_data_dir / "images").glob("*.png"))
    print(f"\nНайдено {len(image_files)} PNG файлов")
    
    cnt = 0
    for img_path in image_files:
        json_path = raw_data_dir / "jsons" / f"{img_path.stem}.json"
#         print(f"\nОбработка файла: {img_path.name}")
#         print(f"Поиск JSON: {json_path}")
        
        if not json_path.exists():
            print(f"Пропускаем {img_path.name}: нет соответствующего JSON файла")
            continue

        try:
            # проверяем, что изображение читается
            img = cv2.imread(str(img_path))
            if img is None:
                print(f"Пропускаем {img_path.name}: невозможно прочитать изображение")
                continue

            # копируем изображение
            new_img_path = images_dir / img_path.name
            shutil.copy(str(img_path), str(new_img_path))
#             print(f"Изображение скопировано в {new_img_path}")

            # читаем JSON
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # создаем YOLO-формат аннотаций
            label_content = []
            img_width = data['image_width']
            img_height = data['image_height']

            for class_name, boxes in data.items():
                if class_name in class_map and isinstance(boxes, list) and boxes:
                    class_id = class_map[class_name]
                    for bbox in boxes:
                        yolo_bbox = convert_to_yolo_format(bbox, img_width, img_height)
                        label_content.append(f"{class_id} {' '.join(map(str, yolo_bbox))}")

            # сохраняем файл с аннотациями
            label_file = labels_dir / f"{img_path.stem}.txt"
            with open(label_file, 'w') as f:
                f.write('\n'.join(label_content))
#             print(f"Создан файл аннотаций: {label_file}")

            valid_pairs.append(img_path.stem)
#             print(f"Пара успешно обработана")
            cnt += 1
            if cnt % 100 == 0:
                print(f"Обработано {cnt} файлов")
            
        except Exception as e:
            print(f"Ошибка при обработке {img_path.name}: {str(e)}")
            continue

    print(f"\nИтоги обработки:")
    print(f"Всего найдено изображений: {len(image_files)}")
    print(f"Успешно обработано пар: {len(valid_pairs)}")

    if not valid_pairs:
        raise ValueError("Не найдено валидных пар изображение-разметка!")

    # создаем train/val split (80/20)
    np.random.shuffle(valid_pairs)
    split_idx = int(len(valid_pairs) * 0.8)
    train_pairs = valid_pairs[:split_idx]
    val_pairs = valid_pairs[split_idx:]

    # сохраняем списки train/val с полными путями
    with open(output_dir / "train.txt", 'w') as f:
        f.write('\n'.join(str(images_dir / f"{name}.png") for name in train_pairs))
    
    with open(output_dir / "val.txt", 'w') as f:
        f.write('\n'.join(str(images_dir / f"{name}.png") for name in val_pairs))

    print(f"\nСоздание файлов со списками:")
    print(f"train.txt: {len(train_pairs)} образцов")
    print(f"val.txt: {len(val_pairs)} образцов")
    
    return len(valid_pairs)

def main():
    # подготовка данных
    print("Подготовка данных...")
    num_samples = prepare_dataset()
    output_dir = Path('/kaggle/working/datasets/prepared_data')

    # создаем конфигурационный файл
    config_dir = Path('/kaggle/working/config')
    config_dir.mkdir(exist_ok=True)
    
    config_content = f"""
path: {str(output_dir)}
train: {str(output_dir / 'train.txt')}
val: {str(output_dir / 'val.txt')}

nc: 12
names: ['title', 'paragraph', 'table', 'picture', 'table_signature', 'picture_signature', 
        'numbered_list', 'marked_list', 'header', 'footer', 'footnote', 'formula']
"""
    
    config_path = config_dir / "doclayout.yaml"
    with open(config_path, 'w') as f:
        f.write(config_content)

if __name__ == "__main__":
    main()


# Обучение модели

Ещё кое-какие установки для исключения ошибок:

In [ ]:
!pip install -U ipywidgets -q
!pip uninstall ray[tune] -y -q

In [ ]:
!pip install doclayout_yolo -q

## Обучение модели на GPU

Основной код обучения модели:

In [ ]:
import os
import gc
from pathlib import Path
from doclayout_yolo import YOLOv10
from huggingface_hub import hf_hub_download
import wandb
import yaml
from datetime import datetime
import torch

def setup_wandb(config_path):
    """Настройка Weights & Biases"""
    with open(config_path) as f:
        data_config = yaml.safe_load(f)
    wandb.init(
        project="DocLayout-YOLO-Training",
        name=f"run_{datetime.now().strftime('%Y%m%d_%H%M')}",
        config={
            "architecture": "YOLOv10",
            "dataset_size": 16000,
            "batch_size": 4,
            "learning_rate": 0.001,
            "epochs": 12,
            "image_size": 800,
            "classes": data_config['names']
        }
    )

def cleanup_gpu_memory():
    """Базовая очистка памяти GPU"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def main():
    cleanup_gpu_memory()
    if torch.cuda.is_available():
        torch.cuda.set_per_process_memory_fraction(0.9)

    # определение путей
    project_root = Path('/kaggle/working')
    raw_data_dir = Path('/kaggle/input/doclayout-raw-data/raw_data')
    output_dir = Path('/kaggle/working/datasets/prepared_data')

    # создание конфигурационного файла
    config_dir = Path('/kaggle/working/config')
    config_dir.mkdir(exist_ok=True)
    config_content = f"""
path: {str(output_dir)}
train: {str(output_dir / 'train.txt')}
val: {str(output_dir / 'val.txt')}
nc: 12
names: ['title', 'paragraph', 'table', 'picture', 'table_signature', 
        'picture_signature', 'numbered_list', 'marked_list', 'header', 
        'footer', 'footnote', 'formula']
"""
    config_path = config_dir / "doclayout.yaml"
    with open(config_path, 'w') as f:
        f.write(config_content)

    setup_wandb(config_path)

    print("Загрузка предобученной модели...")
    try:
        model_path = hf_hub_download(
            repo_id="juliozhao/DocLayout-YOLO-DocStructBench",
            filename="doclayout_yolo_docstructbench_imgsz1024.pt"
        )
    except Exception as e:
        print(f"Ошибка загрузки модели: {e}")
        return

    model = YOLOv10(model_path)

    print("Запуск обучения...")
    results = model.train(
        data=str(config_path),
        epochs=12,
        imgsz=800,
        batch=4,
        workers=4,
        project='DocLayout YOLO',
        name="experiment3",
        exist_ok=True,
        cache=False,
        pretrained=True,
        resume=False,
        verbose=True,
        amp=False, 
        optimizer="Adam",
        lr0=0.001,
        lrf=0.01,
        momentum=0.935,
        weight_decay=0.0005,
        warmup_epochs=3.0,
        warmup_momentum=0.8,
        warmup_bias_lr=0.1,
        box=7.5,
        cls=0.5,
        dfl=1.5,
        plots=True,
        save=True,
        save_period=3,
        multi_scale=False,
        augment=True,
        degrees=2.0,        
        hsv_h=0.2,         
        hsv_s=0.2,       
        hsv_v=0.2,        
        mosaic=0.0,      
        flipud=0.0,         
        fliplr=0.0,         
        patience=10,
        overlap_mask=False
    )
    
    if hasattr(results, 'results_dict'):
        metrics = results.results_dict
        print("\nИтоговая статистика обучения:")
        if 'metrics/mAP50(B)' in metrics:
            print(f"Лучший mAP50: {max(metrics['metrics/mAP50(B)']):.4f}")
        if 'metrics/mAP50-95(B)' in metrics:
            print(f"Лучший mAP50-95: {max(metrics['metrics/mAP50-95(B)']):.4f}")

        wandb.run.summary['best_map50'] = max(metrics.get('metrics/mAP50(B)', [0]))
        wandb.run.summary['best_map50_95'] = max(metrics.get('metrics/mAP50-95(B)', [0]))
        wandb.finish()

    print(f"Обучение завершено. Модель сохранена в {project_root}/runs/train/exp/weights")
    return results

if __name__ == "__main__":
    try:
        cleanup_gpu_memory()
        
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
        
        results = main()
        print("Обучение успешно завершено")
    except Exception as e:
        import traceback
        print(f"Произошла ошибка: {e}")
        print("Полный стек ошибки:")
        print(traceback.format_exc())
    finally:
        cleanup_gpu_memory()


# Результаты

В результате наша модель достигла довольно высоких метрик (их показатели лежат в одной папке с ноутбуком)

---

Для улучшения модели далее сконцентрируемся на следующем:
* Попробуем преобразовать image_size для более точного соотношения сторон
* Добавим метрики **IoU** и **F1 для IoU > 0,5**
* Добавим метрики для отдельных классов
* Будем анализировать эффективность модели в том числе с использованием confusion matrix
* Попробуем использовать другие датасеты для значительного повышения обобщающей способности модели